In [1]:
import os 
import numpy as np 
import os
import re 
import csv 
import math 
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize

# TODO
##### clean data some more 
##### make sure classes even in train, test data 
##### balance class split 


In [2]:

docs = {}
labels = {}
#I only work if you run me in the same folder as the actual text files 

with open('training_labels.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            #print (row) #these are in the order idx,id,score
            line_count += 1
        else:
            #print(row)
            id = int(row[1])
            score= float(row[2])
            labels[id]=score
            line_count += 1
    print(line_count)

for root, dirs, files in os.walk("./"):
    for file in files:
        if file.endswith(".txt"):
            id = re.sub("[^0-9]", "", file)
            id = int(id) 
            path_file = os.path.join(root,file)
            curdir = path_file
            file = open(curdir, 'r') 
            text = file.read()
            
            docs[id] = text

535


In [3]:
train_x,train_y = [],[]
for k in labels:
    try:
        if docs[k] and labels[k]:
            train_x.append(docs[k]) 
            train_y.append(labels[k])
    except KeyError:
        pass

#these two should match 
# print (len(train_x))
# print (len(train_y))


indices = math.floor(len(train_x)*.2)
indices = list(range(0,indices))
test_x = [train_x[x] for x in indices]
test_y = [train_y[x] for x in indices]
train_x = train_x[indices[-1]+1:] 
train_y = train_y[indices[-1]+1:]
print(len(test_x))
print(len(train_x))
print(len(test_x) + len(train_x))

86
348
434


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# def stemming_tokenizer(text):
#     stemmer = PorterStemmer()
#     return [stemmer.stem(w) for w in word_tokenize(text)]
# def vectorize(txt):
#     count_vect = CountVectorizer(tokenizer = stemming_tokenizer, stop_words=stopwords.words('english'),max_features = 5,strip_accents ='ascii')
    
#     vects = count_vect.fit_transform(txt)
#     print(vects.shape) 
#     return vects

def vectorize(txt):
    count_vect = CountVectorizer(stop_words=stopwords.words('english'),max_features = 6,strip_accents ='ascii')
    #print(count_vect.get_feature_names)
    vects = count_vect.fit_transform(txt)
    print(vects.shape) 
    return vects



# def vectorize(txt):
#     vectorizer = TfidfVectorizer(stop_words='english',max_features = 10)
#     vects = vectorizer.fit_transform(txt)
#     print(vects.shape) 
#     return vects

from sklearn.preprocessing import normalize 
def normalize_vect(v):
    return normalize(v)
    


In [6]:
#c = vectorize(train_x)

train_x_vects = vectorize(train_x)
test_x_vects = vectorize(test_x)


(348, 6)
(86, 6)


In [7]:
model = SVC(C=1, cache_size=200, class_weight=None, coef0=1.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=False,
  tol=0.001, verbose=False)

In [8]:
model.fit(train_x_vects, train_y)

SVC(C=1, cache_size=200, class_weight=None, coef0=1.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=False,
  tol=0.001, verbose=False)

In [9]:
predictions = model.predict(test_x_vects)

In [10]:
np.mean(predictions == test_y)

0.7209302325581395

In [19]:
from sklearn.metrics import f1_score, precision_score, recall_score

# print(recall_score(test_y, predictions, average = 'micro'))
# print(precision_score(test_y, predictions, average = 'micro'))
print(f1_score(test_y, predictions, average = 'weighted'))

0.7069208917136116


C:\Users\Erik\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Decision Trees

In [405]:
from sklearn.tree import DecisionTreeClassifier
dt  = DecisionTreeClassifier( random_state = 2, criterion='entropy', splitter = 'best', max_depth=11)
dt.fit(train_x_vects, train_y)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=11,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=2,
            splitter='best')

In [406]:
predi = dt.predict(test_x_vects, test_y)

In [407]:
np.mean(predi == test_y)

0.7906976744186046

In [408]:
print(f1_score(test_y, predi, average = 'weighted'))

0.7986891740873902


## ADABOOST


In [659]:
from sklearn.ensemble import AdaBoostClassifier
#good params base_estimator = dt,n_estimators=5, learning_rate=1.0, algorithm='SAMME.R', random_state=1)

#good params w nltk stop words model = AdaBoostClassifier(base_estimator = dt,n_estimators=14, learning_rate=.2, algorithm='SAMME.R', random_state=1)
ada = AdaBoostClassifier(base_estimator = dt,n_estimators=30, learning_rate=1.9, algorithm='SAMME.R')

In [660]:
ada.fit(train_x_vects, train_y)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=11,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=2,
            splitter='best'),
          learning_rate=1.9, n_estimators=30, random_state=None)

In [661]:
predi = ada.predict(test_x_vects)

In [662]:
np.mean(predi == test_y)

0.7790697674418605

# other stuffs

In [139]:
#max 81.3 adaboost 

In [140]:
# def vectorize(txt):
#     count_vect = CountVectorizer(stop_words=stopwords.words('english'),max_features = 5,strip_accents ='ascii')
#     #print(count_vect.get_feature_names)
#     vects = count_vect.fit_transform(txt)
#     print(vects.shape) 
#     return vects


# from sklearn.tree import DecisionTreeClassifier
# dt  = DecisionTreeClassifier( random_state = 4, criterion='entropy', splitter = 'best', max_depth=None)
# dt.fit(train_x_vects, train_y)


# from sklearn.ensemble import AdaBoostClassifier
# #good params base_estimator = dt,n_estimators=5, learning_rate=1.0, algorithm='SAMME.R', random_state=1)

# #good params w nltk stop words model = AdaBoostClassifier(base_estimator = dt,n_estimators=14, learning_rate=.2, algorithm='SAMME.R', random_state=1)
# model = AdaBoostClassifier(base_estimator = dt,n_estimators=5, learning_rate=1.0, algorithm='SAMME.R', random_state=2)



## Random Forest 

In [555]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 30, random_state = 4, criterion='entropy',  max_depth=13)

In [556]:
forest.fit(train_x_vects, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=13, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=4, verbose=0, warm_start=False)

In [557]:
predi = forest.predict(test_x_vects)

In [558]:
np.mean(predi == test_y)

0.8255813953488372

In [ ]:
#max 82.55

In [663]:
# from sklearn.ensemble import RandomForestClassifier

# forest = RandomForestClassifier(n_estimators = 30, random_state = 4, criterion='entropy',  max_depth=13)